# Managing Device State

Recovering variables from previous notebooks:

In [ ]:
#Hour and minute when this started to create unique names
%store -r unique
#Home directory of the user running this notebook
%store -r home
#Private key file name, for AWS IoT authentication
%store -r private_key
#Certificate file name, for AWS IoT authentication
%store -r certificate_file
#Certificate authority file name, for AWS IoT authentication
%store -r ca_file
#Address to connect using MQTT
%store -r endpoint_address
#The name of the topic to send MQTT messages to
%store -r topic_name
#A sample message payload
%store -r payload
#Thing name
%store -r thing_name

Create the MQTT Shadow client. This client will use a predefined topic structure to manage device state, but it is all plain old MQTT under the hood. See the [Using Shadows](https://docs.aws.amazon.com/iot/latest/developerguide/using-device-shadows.html) documentation page for more information.

In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTShadowClient
client_id="shadow_client_{}".format(unique)
shadow = AWSIoTMQTTShadowClient(client_id)
shadow.configureEndpoint(endpoint_address, 8883)
shadow.configureCredentials(ca_file, private_key, certificate_file)
shadow.configureConnectDisconnectTimeout(600)
shadow.configureMQTTOperationTimeout(10)
shadow.connect()

On top of the client, a shadow handler is used to dispatch messages and callbacks:

In [ ]:
shadow_handler = shadow.createShadowHandlerWithName(thing_name, True)

Let's report the initial device state. Suppose the device was just connected to power and should be standing by with the usual red led.

In [ ]:
initial_state = {
    "power_status": "standby",
    "led_color": "red"
}

import json
shadow_state = {
    "state": {
        "reported" : initial_state
    }
}
shadow_str = json.dumps(shadow_state, indent = 4)
print(shadow_str)

In [ ]:
def update_cb(payload, responseStatus, token):
    payload_json = json.loads(payload)
    print(str(responseStatus))
    print(str(token))
    print(json.dumps(payload_json,sort_keys=True,indent=4))
    

shadow_handler.shadowUpdate(shadow_str, update_cb, 5);

Check your device shadow document on AWS Console:

In [ ]:
print("https://console.aws.amazon.com/iot/home?region=us-east-1#/thing/{}".format(thing_name))


Notice that shadow messages include the metadata required for optimistic conflict resolution and other management information.

Confirm by retrieving the current shadow from AWS IoT:

In [ ]:
def get_cb(payload, responseStatus, token):
    payload_json = json.loads(payload)
    print(json.dumps(payload_json,sort_keys=True,indent=4))
    print(str(responseStatus))
    print(str(token))

shadow_handler.shadowGet(get_cb, 5);

In [ ]:
#Wait for it
import time
time.sleep(1)

Try changing the device state on AWS Console and getting from here again.

Now let's update the desired state. This will let the device know the desired configuration as soon as it is connected and ready to process deltas.

In [ ]:
updated_state = {
    "led_color": "blue",
    "screen_brightness": "0.42"
}

shadow_state = {
    "state": {
        "desired" : updated_state
    }
}
shadow_str = json.dumps(shadow_state, indent=4)
shadow_handler.shadowUpdate(shadow_str, update_cb, 5);
print(shadow_str)

On the device side, we can get the current shadow and either update the deltas or refresh to desired state. Notice that the state now reports not only the desired and reported states but also de differences in ```delta```.

In [ ]:
shadow_handler.shadowGet(get_cb, 5);

With this data your application can always show the correct state to applications and execute the desired changes on the device without managing conectivity and databases.

Speaking of databases, we don't want to loose any data, right?

Let's proceed to the [Storing Telemetry Data](aws-iot-storing-telemetry.ipynb) notebook right now!

# Extra Credit

The MQTT Connection from the shadow client can be used to subscribe directly to the underlying message topics and get notified of state changes:

In [ ]:
deltas_topic = "$aws/things/{}/shadow/update/accepted".format(thing_name)
print(deltas_topic)
mqtt = shadow.getMQTTConnection()
# mqtt.subscribe...